# Final Project Submission
---

## Info

* Student name: **Barto Molina**
* Student pace: **part time**
* Scheduled project review date/time: **TODO**
* Instructor name: **Victor Geislinger**
* Blog post URL: **TODO**

## The Project

Given the data included in the Northwind database, we'll be trying to answer the following question:
> **Does discount amount have a statistically significant effect on the quantity of a product in an order? If so, at what level(s) of discount?**

The schema of the Northwind database can be found below:

![Northwind DB Schema](Northwind_ERD_updated.png "Northwind DB Schema")

## Imports

We'll import the required libraries that will be used throughout the rest of the project:

In [10]:
import sqlite3
import pandas as pd

Let's load the information from the order details into a dataframe

In [40]:
conn = sqlite3.connect("Northwind_small.sqlite")
cur = conn.cursor()

df = pd.read_sql_query("""SELECT CAST (Discount * 100 AS INT) AS Discount, SUM(Quantity) AS Quantity
                       FROM OrderDetail GROUP BY Discount""", conn)

cur.close()
conn.close()

df

,Discount,Quantity
0,0,28599
1,1,2
2,2,4
3,3,5
4,4,1
5,5,5182
6,6,2
7,10,4366
8,15,4456
9,20,4351


We can see that the main discount categories are 0%, 5%, 10%, 15%, 20% and 25%. We'll ignore the rest of the discount categories:

In [42]:
conn = sqlite3.connect("Northwind_small.sqlite")
cur = conn.cursor()

df = pd.read_sql_query("""SELECT CAST (Discount * 100 AS INT) AS Discount, SUM(Quantity) AS Quantity
                       FROM OrderDetail GROUP BY Discount HAVING Quantity > 5""", conn)

cur.close()
conn.close()

df.head(20)

,Discount,Quantity
0,0,28599
1,5,5182
2,10,4366
3,15,4456
4,20,4351
5,25,4349


In [43]:
df.

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x000001C442BA97B8>,
      dtype=object)